## Section 1: Import Required Libraries and Set Up API Keys

In [ ]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
from typing import Dict, List, Any
from datetime import datetime
import random

# Check for OpenAI API key
if 'OPENAI_API_KEY' not in os.environ:
    print("⚠️  WARNING: OPENAI_API_KEY not set. Set it before running agents:")
    print("   export OPENAI_API_KEY='sk-...'")
else:
    print("✓ OpenAI API key detected")

# Import from project
from agents import Agent, MockAgent
from config import ExperimentConfig, AUCTION_SEALED_BID

print("\n✓ All imports successful")

## Section 2: Load Real Ethical Healthcare Vignettes

In [ ]:
# Load the real dataset
df_vignettes = pd.read_csv('Ethical-Reasoning-in-Mental-Health.csv')

print(f"Loaded {len(df_vignettes)} ethical healthcare vignettes")
print(f"\nColumns: {list(df_vignettes.columns)}")
print(f"\nSubcategories: {df_vignettes['subcategory'].unique()[:5]}...")

# Sample 3-5 vignettes for MVP
n_vignettes = 3
sampled_indices = random.sample(range(len(df_vignettes)), n_vignettes)
sample_vignettes = df_vignettes.iloc[sampled_indices].to_dict('records')

print(f"\n✓ Selected {n_vignettes} vignettes for demo:")
for i, vig in enumerate(sample_vignettes, 1):
    print(f"  {i}. {vig['subcategory']}")

## Section 3: Initialize OpenAI Client and Create Agents

In [ ]:
# Initialize 20 agents with different communication styles
n_agents = 20
styles = ['assertive'] * 5 + ['timid'] * 5 + ['calibrated'] * 5 + ['neutral'] * 5
random.shuffle(styles)

# Check if we want to use real OpenAI or mock agents for testing
use_real_openai = True  # Set to False for testing without API calls

agents = []
for i in range(n_agents):
    agent_class = Agent if use_real_openai else MockAgent
    agent = agent_class(
        agent_id=f"agent_{i:02d}",
        communication_style=styles[i],
        budget=1.0
    )
    agents.append(agent)

agent_styles_count = Counter([a.communication_style for a in agents])
print(f"✓ Created {n_agents} agents:")
for style, count in agent_styles_count.items():
    print(f"  - {count} {style} agents")

print(f"\nUsing: {'Real OpenAI API' if use_real_openai else 'Mock agents (no API calls)'}")

## Section 4: Implement Auction Mechanism

In [ ]:
def run_sealed_bid_auction(bids: Dict[str, float]) -> tuple:
    """
    Run a sealed-bid (first-price) auction.
    Winner pays their bid.
    """
    if not bids:
        return None, 0
    winner_id = max(bids, key=bids.get)
    amount_paid = bids[winner_id]
    return winner_id, amount_paid

def run_auction_round(vignette: Dict, agents: List, config: ExperimentConfig) -> Dict[str, Any]:
    """
    Run one complete auction round on a single vignette.
    
    Phases:
    1. Private Assessment: Each agent independently assesses the vignette
    2. Auction: Agents bid for proposer role
    3. Proposal & Interventions: Winner proposes; others can critique for fee
    4. Vote: Agents vote on final answer
    5. Payoff: Distribute rewards and deduct costs
    """
    
    vignette_id = vignette.get('id', 'unknown')
    round_results = {
        'vignette_id': vignette_id,
        'vignette_category': vignette.get('subcategory', ''),
        'agents': defaultdict(dict),
        'bids': {},
        'proposer': None,
        'proposal': None,
        'interventions': defaultdict(list),
        'votes': defaultdict(list),
        'costs_by_agent': defaultdict(float),
    }
    
    # PHASE 1: PRIVATE ASSESSMENT
    print(f"  Phase 1: Private Assessment")
    for agent in agents:
        assessment = agent.assess(vignette)
        round_results['agents'][agent.agent_id]['assessment'] = assessment
    
    # PHASE 2: SEALED-BID AUCTION
    print(f"  Phase 2: Sealed-Bid Auction")
    bids = {}
    for agent in agents:
        assessment = round_results['agents'][agent.agent_id]['assessment']
        bid = agent.bid(vignette, assessment)
        bids[agent.agent_id] = bid
    
    round_results['bids'] = bids
    proposer_id, auction_cost = run_sealed_bid_auction(bids)
    round_results['proposer'] = proposer_id
    round_results['costs_by_agent'][proposer_id] += auction_cost
    
    print(f"    Proposer: {proposer_id} (bid: ${auction_cost:.4f})")
    
    # PHASE 3: PROPOSAL & INTERVENTIONS
    print(f"  Phase 3: Proposal & Optional Critiques")
    proposer_agent = next(a for a in agents if a.agent_id == proposer_id)
    proposer_assessment = round_results['agents'][proposer_id]['assessment']
    proposal_result = proposer_agent.propose(vignette, proposer_assessment)
    round_results['proposal'] = proposal_result['proposal_text']
    round_results['costs_by_agent'][proposer_id] += proposal_result['cost']
    
    print(f"    Proposal: \"{proposal_result['proposal_text'][:100]}...\"")
    print(f"    Proposal cost: ${proposal_result['cost']:.4f}")
    
    # Non-proposers can intervene
    for agent in agents:
        if agent.agent_id != proposer_id:
            assessment = round_results['agents'][agent.agent_id]['assessment']
            intervention = agent.intervene(vignette, proposal_result['proposal_text'], assessment)
            if intervention:
                round_results['interventions'][agent.agent_id] = intervention
                round_results['costs_by_agent'][agent.agent_id] += intervention['cost']
    
    n_interventions = len(round_results['interventions'])
    print(f"    Interventions: {n_interventions} agents critiqued")
    
    # PHASE 4: VOTING
    print(f"  Phase 4: Final Vote")
    options = vignette.get('options', [])
    if isinstance(options, str):
        try:
            options = json.loads(options)
        except:
            options = []
    
    votes = Counter()
    for agent in agents:
        vote = agent.vote(options)
        votes[vote] += 1
        round_results['votes'][agent.agent_id] = vote
    
    consensus_answer = votes.most_common(1)[0][0] if votes else "No consensus"
    consensus_votes = votes[consensus_answer]
    round_results['consensus_answer'] = consensus_answer
    round_results['consensus_votes'] = consensus_votes
    
    expected_answer = vignette.get('expected_reasoning', '')
    correctness = 1.0 if consensus_answer and expected_answer in consensus_answer else 0.5  # Partial credit
    round_results['correctness'] = correctness
    
    print(f"    Consensus: \"{consensus_answer[:50]}...\" ({consensus_votes}/{len(agents)} votes)")
    print(f"    Correctness: {correctness:.1%}")
    
    # PHASE 5: PAYOFF
    print(f"  Phase 5: Payoff Calculation")
    reward_amount = 1.0 if correctness >= 0.8 else 0.5 if correctness >= 0.5 else 0.0
    round_results['reward_pool'] = reward_amount * len(agents)
    
    total_costs = sum(round_results['costs_by_agent'].values())
    total_rewards = 0.0
    
    for agent in agents:
        # Each agent gets: (reward_amount - their_cost)
        agent_cost = round_results['costs_by_agent'][agent.agent_id]
        agent_reward = reward_amount - agent_cost
        round_results['agents'][agent.agent_id]['reward'] = agent_reward
        total_rewards += agent_reward
    
    round_results['total_costs'] = total_costs
    round_results['total_rewards'] = total_rewards
    
    print(f"    Reward per agent: ${reward_amount:.4f}")
    print(f"    Total costs: ${total_costs:.4f}")
    print(f"    Total rewards: ${total_rewards:.4f}")
    
    return round_results

print("✓ Auction functions defined")

## Section 5: Run Multi-Agent Auction Simulation

In [ ]:
# Reset agents for fresh vignettes
for agent in agents:
    agent.reset_for_new_vignette()

print(f"Running {n_vignettes} auction rounds with {n_agents} agents...\n")
print("="*70)

all_round_results = []

for round_num, vignette in enumerate(sample_vignettes, 1):
    print(f"\nRound {round_num}: {vignette['subcategory']}")
    print("-" * 70)
    
    round_result = run_auction_round(vignette, agents, AUCTION_SEALED_BID)
    all_round_results.append(round_result)
    
    # Reset agents for next round
    for agent in agents:
        agent.reset_for_new_vignette()
    
    print()

print("="*70)
print("\n✓ Simulation complete!")

## Section 6: Analyze Agent Behavior and Outcomes

In [ ]:
# Aggregate results
print("\nSimulation Summary")
print("="*70)

total_correctness = sum(r['correctness'] for r in all_round_results) / len(all_round_results)
total_costs = sum(r['total_costs'] for r in all_round_results)
total_rewards = sum(r['total_rewards'] for r in all_round_results)

print(f"Average Correctness:  {total_correctness:.1%}")
print(f"Total Costs (all rounds):  ${total_costs:.4f}")
print(f"Total Rewards (all rounds): ${total_rewards:.4f}")
print(f"Efficiency (Correctness / Cost): {total_correctness / (total_costs + 0.001):.2f}")
print(f"\nKey Insight: Higher total rewards = more strategic, meaningful collaboration")
print(f"            (not just cost reduction, but correctness maintenance)")

# Agent-level analysis
print("\n" + "="*70)
print("Agent-Level Performance by Communication Style")
print("="*70)

agent_stats = defaultdict(lambda: {'total_cost': 0.0, 'total_reward': 0.0, 'count': 0})

for round_result in all_round_results:
    for agent_id, agent_data in round_result['agents'].items():
        # Find agent's style
        agent_obj = next((a for a in agents if a.agent_id == agent_id), None)
        if agent_obj:
            style = agent_obj.communication_style
            cost = round_result['costs_by_agent'][agent_id]
            reward = agent_data.get('reward', 0.0)
            
            agent_stats[style]['total_cost'] += cost
            agent_stats[style]['total_reward'] += reward
            agent_stats[style]['count'] += 1

for style in sorted(agent_stats.keys()):
    stats = agent_stats[style]
    avg_cost = stats['total_cost'] / stats['count'] if stats['count'] > 0 else 0
    avg_reward = stats['total_reward'] / stats['count'] if stats['count'] > 0 else 0
    print(f"\n{style.upper()}:")
    print(f"  Avg Cost:     ${avg_cost:.4f}")
    print(f"  Avg Reward:   ${avg_reward:.4f}")
    print(f"  Net Benefit:  ${avg_reward - avg_cost:.4f}")

## Section 7: Visualization of Results

In [ ]:
# Set up plotting
sns.set_style('whitegrid')
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('LLM Auction Optimization Results', fontsize=16, fontweight='bold')

# Plot 1: Correctness per vignette
correctness_by_round = [r['correctness'] for r in all_round_results]
categories = [r['vignette_category'][:20] for r in all_round_results]

axes[0, 0].bar(range(len(correctness_by_round)), correctness_by_round, color='steelblue', alpha=0.7)
axes[0, 0].set_xticks(range(len(categories)))
axes[0, 0].set_xticklabels(categories, rotation=45, ha='right')
axes[0, 0].set_ylabel('Correctness')
axes[0, 0].set_title('Correctness by Vignette')
axes[0, 0].set_ylim([0, 1.1])
axes[0, 0].axhline(y=total_correctness, color='red', linestyle='--', label='Avg')
axes[0, 0].legend()

# Plot 2: Cost vs Reward per round
costs_by_round = [r['total_costs'] for r in all_round_results]
rewards_by_round = [r['total_rewards'] for r in all_round_results]

x = np.arange(len(all_round_results))
width = 0.35
axes[0, 1].bar(x - width/2, costs_by_round, width, label='Total Costs', alpha=0.7)
axes[0, 1].bar(x + width/2, rewards_by_round, width, label='Total Rewards', alpha=0.7)
axes[0, 1].set_xlabel('Round')
axes[0, 1].set_ylabel('Amount ($)')
axes[0, 1].set_title('Costs vs Rewards by Round')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels([f'R{i+1}' for i in range(len(all_round_results))])
axes[0, 1].legend()

# Plot 3: Agent performance by communication style
styles_list = sorted(agent_stats.keys())
avg_costs = [agent_stats[s]['total_cost'] / agent_stats[s]['count'] for s in styles_list]
avg_rewards = [agent_stats[s]['total_reward'] / agent_stats[s]['count'] for s in styles_list]

x_styles = np.arange(len(styles_list))
axes[1, 0].bar(x_styles - width/2, avg_costs, width, label='Avg Cost', alpha=0.7)
axes[1, 0].bar(x_styles + width/2, avg_rewards, width, label='Avg Reward', alpha=0.7)
axes[1, 0].set_xlabel('Communication Style')
axes[1, 0].set_ylabel('Amount ($)')
axes[1, 0].set_title('Agent Performance by Communication Style')
axes[1, 0].set_xticks(x_styles)
axes[1, 0].set_xticklabels(styles_list, rotation=45, ha='right')
axes[1, 0].legend()

# Plot 4: Voting distribution on last round
last_votes = all_round_results[-1]['votes']
vote_counts = Counter(last_votes.values())

axes[1, 1].pie(vote_counts.values(), labels=[f"{k[:30]}..." for k in vote_counts.keys()], autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title(f'Vote Distribution (Final Round)')

plt.tight_layout()
plt.savefig('auction_results.png', dpi=150, bbox_inches='tight')
print("✓ Results saved to auction_results.png")
plt.show()

## Conclusion

This MVP demonstrates:

1. **Real LLM Agents**: 20 OpenAI-powered agents with distinct communication styles deliberating in real-time
2. **Ethical Reasoning**: Agents reason about real healthcare dilemmas from the Ethical Reasoning in Mental Health dataset
3. **Auction Mechanism**: Sealed-bid auctions create incentives for meaningful contribution and strategic bidding
4. **Total Rewards Metric**: Shows whether mechanism promotes genuine collaboration (high correctness + managed costs) vs silence
5. **Scalability**: Framework ready for token-price sweeps and larger experiments (all 50 vignettes × 20 agents)

**Next Steps**:
- Run token price sweep (0.0001 to 0.01) to find optimal pricing
- Scale to all 50 vignettes
- Compare against baselines (free discussion, turn-taking)
- Analyze which communication styles perform best under different prices